In [1]:
!pip --version

# To install pytorch -------------------------------------------------------------------------------------
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# To install Tensorflow ----------------------------------------------------------------------------------
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

# To install Flax ----------------------------------------------------------------------------------------
!pip install flax

# or to install the latest version of Flax:
!pip install --upgrade git+https://github.com/google/flax.git

# Set up transformers
!pip install git+https://github.com/huggingface/transformers

# Check if installed
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I love you'))"

!pip install transformers[torch]

!pip install accelerate -U

!pip install -q datasets peft evaluate

!pip install opacus

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/google/flax.git to /tmp/pip-req-build-972lyog1
  Running command git clone --filter=blob:none --quiet https://github.com/google/flax.git /tmp/pip-req-build-972lyog1
  Resolved https://github.com/google/flax.git to commit 85dfad242e56098849dbf05e7e4657b3a40820f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for flax: filename=flax-0.8.0-py3-none-any.whl size=635291 sha256=63a3876cd4ee678dfb57ce2c9da89a852bc69bce93a3a419bceb85caea8ccf30
  Stored in directory: /tmp/pi

In [2]:
from transformers import (AutoModelForSequenceClassification,AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer )
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
    PromptTuningConfig,
)
from peft import ( get_peft_config, get_peft_model, get_peft_model_state_dict, set_peft_model_state_dict, PeftModelForSeq2SeqLM, PeftType,PromptEncoderConfig,PeftModelForSequenceClassification)
from peft import PromptEmbedding, PromptTuningConfig
from datasets import load_dataset
import evaluate
from transformers import AdamW
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from datasets import Dataset
import opacus
from opacus import PrivacyEngine
from opacus.grad_sample import GradSampleModule
from opacus.validators import ModuleValidator



dataset = load_dataset("glue", "mnli")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=3) # as output 0 or 1
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny", padding_side = "right")
model = model.to('cuda')
# optimizer = AdamW(model.parameters(), lr= 0.005, eps = 1e-8)


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [3]:
config = PromptTuningConfig(
    peft_type="PROMPT_TUNING",
    task_type="TEXT_PAIR",
    num_virtual_tokens= 20,
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="Classify the given question and sentence pair as entailment, neutral or contradiction",
    tokenizer_name_or_path="prajjwal1/bert-tiny",
)

model = PeftModelForSequenceClassification(model, config)
# model = get_peft_model(model, config)
print(model.print_trainable_parameters())

total_params = 0
trainable_params = 0

trainable_layers = [model.prompt_encoder, model.classifier]
for p in model.parameters():
        p.requires_grad = False
        total_params += p.numel()

l = []
for layer in trainable_layers:
    for p in layer.parameters():
        l.append(p)
        p.requires_grad = True
        trainable_params += p.numel()

optimizer = AdamW(l , lr= 0.005, eps = 1e-8)

print("total:",total_params)
print("trainable_param",trainable_params)


trainable params: 2,947 || all params: 4,389,254 || trainable%: 0.06714124997095178
None
total: 4389254
trainable_param 3334


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
config = PromptTuningConfig(
    peft_type="PROMPT_TUNING",
    task_type="TEXT_PAIR",
    num_virtual_tokens= 20,
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="Classify the question-sentence pair as entailment, neutral or contradiction",
    tokenizer_name_or_path="prajjwal1/bert-tiny",
)

# peft_config = PromptTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=60)
# model = get_peft_model(model, peft_config)


model = PeftModelForSequenceClassification(model, config)
model = get_peft_model(model, config)
print(model.print_trainable_parameters())


model = ModuleValidator.fix(model)
model = GradSampleModule(model)

optimizer = AdamW(model.parameters() , lr= 0.005, eps = 1e-8)



In [4]:
# Assuming the dataset has a 'train' split, modify this according to your dataset's splits
data_split = dataset['train']

# Convert the dataset split to a pandas DataFrame for easier splitting
df = data_split.to_pandas()
df = df[0:150000]

# Split the dataset into train and test sets using train_test_split from sklearn
# df1, df2 = train_test_split(df, test_size = 0.5, random_state = 42)
train_df, test_df = train_test_split(df , test_size=0.2, random_state=42)

# Convert the splits back to datasets
train_dataset = train_df.reset_index(drop=True)
test_dataset = test_df.reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

x_train1 = list(train_dataset["premise"])
x_train2 = list(train_dataset["hypothesis"])
y_train = list(train_dataset["label"])

x_test1 = list(test_dataset["premise"])
x_test2 = list(test_dataset["hypothesis"])
y_test = list(test_dataset["label"])

X_train_tokenized = tokenizer(x_train1, x_train2, padding=True, truncation=True, max_length = 256)
# X_val_tokenized = tokenizer(x_validation, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(x_test1, x_test2, padding=True, truncation = True, max_length = 256)


class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to('cuda') for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx]).to('cuda')
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
# val_dataset = Dataset(X_val_tokenized, y_validation)
test_dataset = Dataset(X_test_tokenized, y_test)

train_dataloader = DataLoader(train_dataset, batch_size = 32, drop_last = True)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size = 32, drop_last = True)
# val_dataloader = DataLoader(val_dataset, batch_size = 500)

In [ ]:
model.train()
privacy_engine = PrivacyEngine()
model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    target_delta= 1 / len(train_dataloader),
    target_epsilon= 100,
    epochs= 3,
    max_grad_norm = 0.1,
)

In [5]:
def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()
    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:

        with torch.no_grad():
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss, logits = outputs[:2]

            log = logits.cpu()
            log = log.numpy()

            preds = np.argmax(log, axis=1)
            labels = batch['labels'].detach().cpu().numpy()

            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

model.train()

# Training loop
optimizer.zero_grad()  # Explicitly zero the gradient buffers

for epoch in range(20):  # Number of epochs
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

        logits = outputs[1]
        predictions = torch.argmax(logits, dim = -1)

    eval_loss, eval_accuracy = evaluate(model)
    print("eval loss",eval_loss)
    print("accuracy: ",eval_accuracy)
    print("ends")

    model.train()  # Set the model back to training mode


eval loss 1.0414179412125268
accuracy:  0.452741462113127
ends
eval loss 1.0355054524308716
accuracy:  0.46531483457844186
ends
eval loss 1.0202914447133036
accuracy:  0.46905016008537886
ends
eval loss 1.0219240636428621
accuracy:  0.473352454642476
ends
eval loss 1.025496067779423
accuracy:  0.4781216648879402
ends
eval loss 1.0271772253729619
accuracy:  0.47261872998932764
ends
eval loss 1.0254525714012195
accuracy:  0.46985058697972254
ends
eval loss 1.0165245767082451
accuracy:  0.47872198505869795
ends
eval loss 1.0240305208600065
accuracy:  0.4715181430096051
ends
eval loss 1.0331994468145462
accuracy:  0.4707177161152615
ends
eval loss 1.0277174106911382
accuracy:  0.46571504802561364
ends
eval loss 1.0423814214153757
accuracy:  0.4569770544290288
ends
eval loss 1.0132760888993295
accuracy:  0.48022278548559233
ends
eval loss 1.0259628830877159
accuracy:  0.46578175026680896
ends
eval loss 1.0171122545239255
accuracy:  0.47225186766275346
ends
eval loss 1.020117152335168
accura